In [1]:
import torch
torch.cuda.device_count()

4

In [2]:
from glob import glob
import os
from typing import Sequence
import cucim

scan_path = "/data2/projects/DigiStrudMed_sklein/DigiStrucMed_Bräsen/all_data"
image_paths: list = sorted(glob(os.path.join(scan_path , "*.svs")))
image_paths = [os.path.basename(i).removesuffix(".svs") for i in image_paths]
images = [i.split("-", maxsplit=2)[-1] for i in image_paths]
images = [(i[:3], i[4:7], i[7:]) for i in images]



In [3]:
import pandas as pd

path = "/data2/projects/DigiStrudMed_sklein/DigiStrucMed_Bräsen/RCC-Masterfile_20211124.xlsx"

raw_data = pd.read_excel(
    path, 
    sheet_name="Proben RCC, PD-L1, Metastas (2", 
    header=0, 
    index_col=0,
    nrows=178,
    usecols="A,N,O,GQ,GR,GS,K"
    )
raw_data = raw_data[raw_data.index.notnull()]
raw_data.index = [case[-3:] for case in raw_data.index.astype(str)]

FileNotFoundError: [Errno 2] No such file or directory: '/data2/projects/DigiStrudMed_sklein/DigiStrucMed_Bräsen/RCC-Masterfile_20211124.xlsx'

In [ ]:
result = {}
for case, stain, idk in images:
  if case in result.keys():
      if stain in result[case]:
        result[case][stain].append(idk)
      else:
        result[case][stain] = [idk]
  else:
    result[case] = {stain: [idk]}

### Comparison of images and clinical data

#### Amount of patients

In [ ]:
len(result.keys())

194

In [ ]:
len(raw_data.index)

176

### Tabular data missing

In [ ]:
set(result.keys()).difference(set(raw_data.index))

{'007',
 '014',
 '024',
 '029',
 '035',
 '036',
 '041',
 '051',
 '065',
 '091',
 '097',
 '100',
 '102',
 '104',
 '153',
 '180',
 '182',
 '183'}

### case IDs

In [ ]:
result.keys()

dict_keys(['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '1

### occurring stains

In [ ]:
all_stains = [item for sublist in [case.keys() for case in result.values()] for item in sublist]

In [ ]:
from collections import Counter

print(f"stain: count - {dict(Counter(all_stains))}")

stain: count - {'001': 188, '016': 191, '019': 183, '041': 176, '042': 182}


### Number of caes with less than X stains

In [ ]:
for i in range(6, 1, -1):
    print(f"Number of Patients with less than {i} stains: {len([key for key, value in result.items() if len(value) < i])}")


Number of Patients with less than 6 stains: 194
Number of Patients with less than 5 stains: 18
Number of Patients with less than 4 stains: 18
Number of Patients with less than 3 stains: 10
Number of Patients with less than 2 stains: 4


### Cases where a stain has more than one file

In [ ]:
[(key, value) for case in result.values() for key, value in case.items() if len(value) > 1]

[('019', ['~B', '~E']),
 ('019', ['~B', '~E']),
 ('019', ['~B', '~E']),
 ('019', ['~B', '~E']),
 ('019', ['~B', '~E']),
 ('016', ['~E', '~H']),
 ('019', ['~B', '~E']),
 ('019', ['~B', '~E']),
 ('019', ['~B', '~E']),
 ('019', ['~B', '~E']),
 ('019', ['~B', '~H']),
 ('016', ['~B', '~E']),
 ('016', ['flipped', '~B']),
 ('019', ['flipped', '~B']),
 ('041', ['neu', '~C']),
 ('042', ['flipped', '~B']),
 ('016', ['~E', '~H']),
 ('019', ['~B', '~K']),
 ('019', ['~AC', '~Q']),
 ('019', ['~E', '~H', '~K']),
 ('041', ['~C', '~D_Hemaneu']),
 ('016', ['~B', '~E']),
 ('019', ['~B', '~E']),
 ('001', ['_2nd~C', '~B']),
 ('001', ['_2nd~F', '~B'])]

## Findings

- 019neu
    - This dir contains files that also exit in the other 019 dir. Probably use the new ones (I assume the old ones were incorrect and had to be re-done) 
- Missinng Stains
    - Certain Patients lack stains found in others. 
- Missing Patients
    - The tabular Data lacks Patients that have images available, ignore those? 
- Stains codes
    - Hämalaun (023) is missing, but instead we have 041 now. What to do? 
- "redundant" files
    - Some Cases/patients have multiple files (2 at most) with differing suffix. There seems to be two cases 
- tilde + Capital letter
    - I think certain backup tools attach `~` characters, but why is only a subset duplicated. The different Capital letters are `B`, `E`, `H`  
- divergence from naming scheme
    - Some files differ by having special suffixes. I am not sure what those mean. `neu`, `_2nd~`, `*_Hemanu`, `flipped` 

## Check qupath project files

These files specify which svs files are actually needed

In [7]:
scan_path = '/data2/projects/DigiStrudMed_sklein/DigiStrucMed_Braesen/all_data'

In [8]:
import json

# recursively find all .qpproj files in a directory
gproj_list: list[str] = glob(os.path.join(scan_path+"/..", "**", "*.qpproj"), recursive=True)
[(it.split("/", maxsplit=6))[-1] for it in gproj_list]

['../QuPathProjects/RCC-MaskenБbertragung CD4_FoxP3 N. Vasileiadis - Kopie/project.qpproj',
 '../QuPathProjects/RCC-MaskenБbertragung CD4_FoxP3 N. Vasileiadis - Kopie/project.backup.qpproj',
 '../QuPathProjects/hand-editted/RCC-Maskenubertragung-CD8_CD20-N.Vasileiadis/project.qpproj',
 '../QuPathProjects/hand-editted/RCC-Maskenubertragung-CD4_FoxP3-N.Vasileiadis/project.qpproj',
 '../QuPathProjects/hand-editted/RCC-Maskenubertragung-CD204-N.Vasileiadis/project.qpproj',
 '../QuPathProjects/hand-editted/RCC-Maskenubertragung-CD68-N.Vasileiadis/project.qpproj',
 '../QuPathProjects/RCC-MaskenБbertragung CD68 N. Vasileiadis - Kopie/project (2).backup.qpproj',
 '../QuPathProjects/RCC-MaskenБbertragung CD68 N. Vasileiadis - Kopie/project.qpproj',
 '../QuPathProjects/RCC-MaskenБbertragung CD68 N. Vasileiadis - Kopie/project.backup.qpproj',
 '../QuPathProjects/RCC-MaskenБbertragung CD8_CD20 N. Vasileiadis - Kopie/project.qpproj',
 '../QuPathProjects/RCC-MaskenБbertragung CD8_CD20 N. Vasileiadis

In [5]:
gproj_dicts = [json.load(open(i)) for i in gproj_list]
idx_i = 1
print(gproj_dicts[idx_i]["uri"])

svs_files: list[str] = [item['serverBuilder']["uri"] for idx in range(len(gproj_dicts)) for item in gproj_dicts[idx]["images"]]
svs_files: list[str] = [item['serverBuilder']["uri"] for item in gproj_dicts[idx_i]["images"]]

svs_cases: list[str] = [os.path.basename(it).removesuffix(".svs") for it in svs_files]

IndexError: list index out of range

In [ ]:
sorted(image_paths[:10])

['RCC-TA-001.001~C',
 'RCC-TA-001.016~B',
 'RCC-TA-001.019~B',
 'RCC-TA-001.019~E',
 'RCC-TA-001.041~B',
 'RCC-TA-001.042~B',
 'RCC-TA-002.001~C',
 'RCC-TA-002.016~B',
 'RCC-TA-002.019~B',
 'RCC-TA-002.019~E']

In [ ]:
sorted(svs_cases[:10])

['RCC-TA-001.016~B',
 'RCC-TA-002.016~B',
 'RCC-TA-003.016~B',
 'RCC-TA-004.016~B',
 'RCC-TA-005.016~B',
 'RCC-TA-006.016~B',
 'RCC-TA-007.016~E',
 'RCC-TA-008.016~E',
 'RCC-TA-009.016~E',
 'RCC-TA-011.016~H']

In [ ]:
set(svs_cases).difference(set(image_paths))

{'M1-101.016~B',
 'M1-113.016~B',
 'M1-119.016~B',
 'M1-136.016~B',
 'M1-151.016~B',
 'M1-156.016~B',
 'M1-159.016~B',
 'M1-161.016flipped',
 'M1-162.016~B',
 'M1-163.016~B',
 'M1-165.016~B',
 'M1-166.016~B',
 'M1-167.016~B',
 'M1-173.016~B',
 'M2-136.016~B',
 'M2-151.016~B',
 'M2-162.016~C',
 'M2-173.016~B',
 'M3-173.016~B',
 'RCC-TA-063.016~E',
 'RCC-TA-066.016~E',
 'RCC-TA-068.016~H',
 'RCC-TA-069.016~E'}

## cases with missing surv times

In [ ]:
import numpy as np
cases_w_images = np.asarray(images)[:, 0]
no_tab_data: list[str] = ['067', '066', '077', '137', '134', '150', '152', '155', '168', '196']
[case in cases_w_images for case in no_tab_data]

In [16]:
scan_path:str = "/data2/projects/DigiStrudMed_sklein/DigiStrucMed_Braesen/"
gproj_list: list[str] = glob(os.path.join(scan_path, "**", "*.qpproj"), recursive=True)
[(it.split("/", maxsplit=6))[-1] for it in gproj_list]

['RCC-MaskenБbertragung CD4_FoxP3 N. Vasileiadis - Kopie/project.qpproj',
 'RCC-Maskenubertragung-CD204-N.-Vasileiadis-Kopie/project.qpproj',
 'RCC-MaskenБbertragung CD68 N. Vasileiadis - Kopie/project.qpproj',
 'RCC-MaskenБbertragung CD8_CD20 N. Vasileiadis - Kopie/project.qpproj']

In [19]:
gproj_list

['/data2/projects/DigiStrudMed_sklein/DigiStrucMed_Bräsen/QuPathProjects/RCC-MaskenБbertragung CD4_FoxP3 N. Vasileiadis - Kopie/project.qpproj',
 '/data2/projects/DigiStrudMed_sklein/DigiStrucMed_Bräsen/QuPathProjects/RCC-Maskenubertragung-CD204-N.-Vasileiadis-Kopie/project.qpproj',
 '/data2/projects/DigiStrudMed_sklein/DigiStrucMed_Bräsen/QuPathProjects/RCC-MaskenБbertragung CD68 N. Vasileiadis - Kopie/project.qpproj',
 '/data2/projects/DigiStrudMed_sklein/DigiStrucMed_Bräsen/QuPathProjects/RCC-MaskenБbertragung CD8_CD20 N. Vasileiadis - Kopie/project.qpproj']

In [28]:
gproj_list[1]

'/data2/projects/DigiStrudMed_sklein/DigiStrucMed_Bräsen/QuPathProjects/RCC-Maskenubertragung-CD204-N.-Vasileiadis-Kopie/project.qpproj'

In [30]:
idx_i = 3

gproj_dicts = [json.load(open(i)) for i in [gproj_list[idx_i]]]
print(gproj_dicts[0]["uri"])

svs_files: list[str] = [item['serverBuilder']["uri"] for idx in range(len(gproj_dicts)) for item in gproj_dicts[idx]["images"]]
# svs_files: list[str] = [item['serverBuilder']["uri"] for item in gproj_dicts[idx_i]["images"]]

svs_cases: list[str] = [os.path.basename(it).removesuffix(".svs") for it in svs_files]

file:///data2/projects/DigiStrudMed_sklein/DigiStrucMed_Bräsen/QuPathProjects/RCC-MaskenБbertragung CD8_CD20 N. Vasileiadis - Kopie/project.qpproj
